In [1]:
import torch
import numpy as np
from torchvision import transforms
from custom_transforms import  NormalizeSpectrogram, ToTensor, InterpolateSpectrogram
from SearchNet import SearchNet

def predict_single_spectrogram(spectrogram_array, DEVICE):
    # Apply transforms individually to handle the different input types
    # NormalizeSpectrogram expects numpy array
    normalize = NormalizeSpectrogram()
    normalized_spectrogram = normalize(spectrogram_array)
    
    # ToTensor expects numpy array and returns tensor
    to_tensor = ToTensor()
    tensor_spectrogram = to_tensor(normalized_spectrogram)
    
    # InterpolateSpectrogram expects tensor
    interpolate = InterpolateSpectrogram()
    interpolated_spectrogram = interpolate(tensor_spectrogram)
    
    # Add batch dimension and move to device
    input_tensor = interpolated_spectrogram.unsqueeze(0).to(DEVICE)
    # Initialize and load model
    model = SearchNet(0.10, 1.16, 0.75, 0.75).to(DEVICE)
    model.load_state_dict(torch.load("final_model_final_model"))#, map_location=torch.device('cpu')))
    model.eval()

    #Make prediction
    with torch.no_grad():
        prediction = model(input_tensor)
    
    return prediction.squeeze().cpu().numpy()

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

spectrogram_array = np.load(r"ball_hit_spectrograms.npy")

prediction = predict_single_spectrogram(spectrogram_array, DEVICE)

print(f'Predicted Radial Velocity: {prediction:.3f}')

C:\Users\gtoft\AppData\Local\Temp\ipykernel_19720\3679228442.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("final_model_final_model")

Predicted Radial Velocity: -106.579


# Notes

- Radial velocity is negative seeing as the ball is hit towards the radars
- This is synthetic data (image can be seen synthetic_image.png) with no GT but for context, the prediction is more than what 99.999% of players are able to achieve